In [70]:
import tensorflow as tf
import numpy as np
from json import load
from spacy.util import minibatch, compounding
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.preprocessing import normalize
import re
from spacy.training.example import Example
nlp = spacy.load("ru_core_news_md")


In [71]:
with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    discription_of_event = load(js)
ner = nlp.get_pipe("ner")
punctuation = list(map(lambda a: a, punctuation)) + \
    ['(', ')', '"', '«', '»', "'", "✅"]


In [72]:
def to_describer(text, ent):
    return (text, {"entities": ent})


In [73]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data


In [82]:
TRAIN_DATA = []
for events in discription_of_event:
    description = discription_of_event[events]["описание"]
    new_skills = discription_of_event[events]["new_skills"]
    for sentence in description.replace('\n', ' ').replace('✅', ' ').split("."):
        for skill in new_skills:
            if skill in sentence:
                TRAIN_DATA.append(to_describer(sentence, [(position.start(), position.end()-1, "type") for position in re.finditer(skill, sentence)]))
        
len(TRAIN_DATA)


60

In [83]:
TRAIN_DATA = trim_entity_spans(TRAIN_DATA)


In [84]:
for i in TRAIN_DATA:
    if 'Как найти исполнительного директора и передать' in i[0]:
        print(i)
        print(i[0][i[1]['entities'][0][0]:i[1]['entities'][0][1]])


['    Как найти исполнительного директора и передать оперативное управление', {'entities': [[8, 39, 'type']]}]
найти исполнительного директора
['    Как найти исполнительного директора и передать оперативное управление', {'entities': [[42, 72, 'type']]}]
передать оперативное управлени


In [85]:
spacy.training.offsets_to_biluo_tags(nlp.make_doc(
    'Как найти исполнительного директора и передать оперативное управление'), [(8, 39, 'type')])


['O', '-', '-', '-', '-', '-', 'O', 'O']

In [86]:
nlp.make_doc(
    'Как найти исполнительного директора и передать оперативное управление')


Как найти исполнительного директора и передать оперативное управление

In [87]:
for i in TRAIN_DATA:
    if 'исполнительного директора и передать оперативное' in i[0]:
        print(nlp.tokenizer.explain(i[0]))

[('TOKEN', 'Как'), ('TOKEN', 'найти'), ('TOKEN', 'исполнительного'), ('TOKEN', 'директора'), ('TOKEN', 'и'), ('TOKEN', 'передать'), ('TOKEN', 'оперативное'), ('TOKEN', 'управление')]
[('TOKEN', 'Как'), ('TOKEN', 'найти'), ('TOKEN', 'исполнительного'), ('TOKEN', 'директора'), ('TOKEN', 'и'), ('TOKEN', 'передать'), ('TOKEN', 'оперативное'), ('TOKEN', 'управление')]


In [89]:
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# shuffle(TRAIN_DATA)

losses = {}
for batch in spacy.util.minibatch(TRAIN_DATA, size=compounding(30.0, 50, 5.5)):
    for text, annotations in batch:
        doc = nlp(text)
        example = Example.from_dict(doc, annotations)
        # print(losses)
        nlp.update([example], losses=losses, drop=0.0001)


ValueError: [E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. You can also use the `debug data` command to validate your JSON-formatted training data. For details, run:
python -m spacy debug data --help

In [90]:
# doc1 = nlp.make_doc(
#     'Как найти исполнительного директора и передать оперативное управление'.lower())
# doc1.ents
for i in TRAIN_DATA:
    print(nlp(i[0].lower()).ents)

(антон петроченко,)
(александр гунгер,)
(алексей бондаренко,)
(российской федерации, ано)
(тпп, омской области)
(тпп, омской области)
(тпп, омской области)
(тпп, омской области)
(тпп, омской области)
(тпп, омской области)
()
()
()
()
()
()
()
()
()
()
(рф, ржд, роскосмоса)
()
()
()
()
()
(кристина казазаева, aiesec)
(москвы,)
(москвы,)
(алина судоплатова, проведёт)
()
()
()
()
()
(москвы, ооо "спотзания"- центра дополнительного образования)
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
